In [1]:
import pandas as pd
import numpy as np
import huggingface_hub
import torch
import transformers

In [2]:
huggingface_hub.login()

In [3]:
f = open('10288320_anomaly_rm.txt')
data = f.readlines()
del data[0]

In [4]:
stopword = 'a, are, an, the, and, or, but, if, then, because, about, above, after, all, also, although, am, an, and, any, as, at, be, because, been, before, being, between, both, but, by, came, can, come, could, did, do, does, each, else, for, from, get, got, had, has, have, he, her, here, him, himself, his, how, i, if, in, into, is, it, its, just, like, make, many, me, might, more, most, much, must, my, never, no, nor, not, now, of, on, only, or, other, our, out, over, said, same, see, should, since, so, some, still, such, take, than, that, the, their, them, then, there, these, they, this, those, through, to, too, under, up, use, very, want, was, way, we, well, were, what, when, where, which, while, who, will, with, would, you, your'.replace(' ','').split(',')

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
def preprocess_function(input_text):
    return tokenizer(input_text, truncation=True)

In [7]:
preprocess_function(data[0].replace('\n','').lower().split('\t')[1])

{'input_ids': [101, 2058, 1018, 2454, 4841, 4897, 2039, 15114, 2005, 18168, 2594, 4948, 1011, 9416, 2522, 17258, 23715, 2015, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
library_headline = dict()
library_short_description = dict()
classes = []
class_label_dict = dict()
label_class_dict = dict()

count=0
for i in data[:int(len(data)*0.7)]:
    token = i.replace('\n','').lower().split('\t')
    if token[2] not in classes:
        classes.append(token[2])
        label_class_dict[count] = token[2]
        class_label_dict[token[2]] = count
        count = count + 1

for i in classes:
    library_headline[i] = dict()
    library_short_description[i] = dict()

In [9]:
import evaluate
accuracy = evaluate.load("accuracy")

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(classes), id2label=label_class_dict, label2id=class_label_dict
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [11]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
def preprocess_function(input_text):
    return tokenizer(input_text, truncation=True)

In [14]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.data = data[:int(len(data)*0.7)]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        token = self.data[idx].replace('\n','').lower().split('\t')
        x = preprocess_function(token[1]+' '+token[3])
        y = class_label_dict[token[2]]
        return {'text':token[1]+' '+token[3], 'label':y, 'input_ids':x['input_ids'], 'attention_mask':x['attention_mask']}
    #dict_keys(['text', 'label', 'input_ids', 'attention_mask'])

In [15]:
class myDataset_eval(torch.utils.data.Dataset):
    def __init__(self):
        self.data = data[int(len(data)*0.7):]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        token = self.data[idx].replace('\n','').lower().split('\t')
        x = preprocess_function(token[1]+' '+token[3])
        y = class_label_dict[token[2]]
        return {'text':token[1]+' '+token[3], 'label':y, 'input_ids':x['input_ids'], 'attention_mask':x['attention_mask']}

In [16]:
train_dataset = myDataset()
eval_dataset = myDataset_eval()

In [17]:
from datasets import load_dataset
def testf(input_text):
    return tokenizer(input_text['text'], truncation=True)

imdb = load_dataset("imdb")
tokenized_imdb = imdb.map(testf, batched=True)

Found cached dataset imdb (C:/Users/naya5/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\naya5\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-1884e36b11240839.arrow
Loading cached processed dataset at C:\Users\naya5\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-2b718bfe87966f1e.arrow
Loading cached processed dataset at C:\Users\naya5\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-e3a934cb14f21add.arrow


In [18]:
tokenized_imdb['train'][0].keys()

dict_keys(['text', 'label', 'input_ids', 'attention_mask'])

In [19]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    #train_dataset=tokenized_imdb['train'],
    eval_dataset=eval_dataset,
    #eval_dataset=tokenized_imdb['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train()

c:\Users\naya5\Desktop\works\2023\nlp\doc_classifier\my_awesome_model is already a clone of https://huggingface.co/gkook/my_awesome_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [20]:
trainer.train()

c:\Users\naya5\anaconda3\envs\py38\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/18742 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
